In [125]:
import fastbook
from fastai.vision.all import *
from fastbook import *
matplotlib.rc('image', cmap='Greys')
import warnings
warnings.filterwarnings("ignore")

In [107]:
def get_paths(paths):
    lst = []
    for path in paths.ls().sorted():
        lst.append(path.ls().sorted())
    zeros, ones, twos, threes = lst[0], lst[1], lst[2], lst[3]
    fours, fives, sixes = lst[4], lst[5], lst[6]
    sevens, eights, nines = lst[7], lst[8], lst[9]
    return [zeros, ones, twos, threes, fours, fives, sixes, sevens, eights, nines]

def get_tensors(numbers):
    tensors = [tensor(Image.open(o)) for o in numbers]
    stacked_tensors = torch.stack(tensors).float()/255
    return stacked_tensors

def all_stacked(numbers):
    stacked_lst = []
    for number in numbers:
        stacked_lst.append(get_tensors(number))
    return stacked_lst

def length_list(stacked_lst):
    length_lst = []
    for number in stacked_lst:
        length_lst.append(number.shape[0])
    return length_lst

def split_dset(stacked_lst, length_lst):
    x = torch.cat(stacked_lst).view(-1, 28*28)
    y = tensor([0]*length_lst[0] + [1]*length_lst[1] +
                    [2]*length_lst[2] + [3]*length_lst[3] + [4]*length_lst[4] + 
                    [5]*length_lst[5] + [6]*length_lst[6] + [7]*length_lst[7] + 
                    [8]*length_lst[8] + [9]*length_lst[9]).unsqueeze(1)
    y = torch.argmax(y, axis=1)
    return list(zip(x,y))

def dset_validset():
    path = untar_data(URLs.MNIST)
    Path.BASE_PATH = path
    numbers_train, numbers_valid = get_paths(path/'training'), get_paths(path/'testing')
    stacked_lst_train, stacked_lst_valid = all_stacked(numbers_train), all_stacked(numbers_valid)
    length_lst_train, length_lst_valid = length_list(stacked_lst_train), length_list(stacked_lst_valid)
    dset = split_dset(stacked_lst_train, length_lst_train)
    valid_dset = split_dset(stacked_lst_valid, length_lst_valid)
    return dset, valid_dset

def create_dataloader():
    dset, valid_dset = dset_validset()
    dl, valid_dl = DataLoader(dset, batch_size=256), DataLoader(valid_dset, batch_size=256)
    return dl, valid_dl

In [115]:
dl, valid_dl = create_dataloader()
dls = DataLoaders(dl, valid_dl)

In [119]:
simpleNet = nn.Sequential(nn.Linear(28*28, 30),
                         nn.ReLU(),
                         nn.Linear(30, 15),
                         nn.ReLU(),
                         nn.Linear(15, 5),
                         nn.ReLU(),
                         nn.Linear(5,10))

In [120]:
learn = Learner(dls, simpleNet, opt_func=Adam,
               loss_func=F.cross_entropy, metrics=error_rate)

In [121]:
learn.fit(2, 0.01)

epoch,train_loss,valid_loss,error_rate,time
0,0.417731,0.229246,0.000000,00:01
1,0.065637,0.044297,0.000000,00:00


In [128]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path
# dls = ImageDataLoaders.from_folder(path)
path

Path('.')

In [130]:
path = untar_data(URLs.MNIST_SAMPLE)
dls = ImageDataLoaders.from_folder(path)

In [132]:
learn = cnn_learner(dls, resnet34, pretrained=False, loss_func=F.cross_entropy, metrics=accuracy)

In [134]:
learn.fit_one_cycle(5, 0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.489787,85.285561,0.011776,00:11
1,0.199112,0.063998,0.997056,00:11
2,0.045694,13.665556,0.997547,00:11
3,0.094295,813.051331,0.962218,00:10
4,0.019690,0.020023,0.996565,00:10
